In [1]:
###########################################
# Google doc and ORB setup (run once)
###########################################

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
 
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("Xilinx_Hackathon_2017").sheet1


###### ORB
# imports for ORB
import numpy as np
import cv2

# Initiate ORB detector
orb = cv2.ORB_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

import string
import re
####################################
######   Configurable items:  ######
####################################
# arbitary number. Fine tune once you get here
thres_hold = 250


In [3]:
#TODO: receive the cropped face from the webcam

In [4]:
# New face from the webcam
#TODO update this variable to reference the webcam image
image = '1.JPG'

# Find the Key point
img = cv2.imread(image,0)          # queryImage

# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img,None)
#kp2, des2 = orb.detectAndCompute(img2,None)

# TEST CODE populate the speadsheet with a result
image = '2.JPG'
img = cv2.imread(image,0)          # queryImage
kp2, des2 = orb.detectAndCompute(img,None)


sheet.append_row('')
col_in = 1
row_in = sheet.row_count

collumn_list = []
des_cnt = 0
des_per_cell = 100
for collumn in des2:
    print (des_cnt)
    
    collumn_list.append(collumn.tolist())
    # Append des_per_cell # of collumns into a single collumn and upload
    if des_cnt >= des_per_cell -1:
        print ("cloud")
        sheet.update_cell(row_in, col_in, collumn_list)
        collumn_list = []
        col_in += 1
        des_cnt = 0
    else:
        des_cnt += 1
        

In [5]:
###########################################
# Read from the Google doc
###########################################

In [6]:
# Extract and print all of the values
all_faces_str = sheet.get_all_values()
print(all_faces_str)

[['Filler', '', '', '', ''], ['[[138, 222, 118, 190, 247, 242, 191, 254, 191, 206, 222, 183, 108, 255, 191, 69, 255, 251, 191, 199, 244, 223, 47, 255, 127, 127, 254, 203, 183, 221, 189, 223], [30, 159, 254, 247, 239, 218, 63, 248, 191, 254, 247, 167, 95, 255, 171, 91, 247, 255, 255, 99, 240, 209, 169, 238, 255, 247, 111, 135, 167, 255, 173, 115], [175, 118, 231, 223, 101, 191, 255, 123, 95, 234, 125, 246, 249, 222, 190, 188, 255, 233, 183, 175, 239, 246, 125, 223, 38, 95, 183, 166, 127, 254, 249, 245], [170, 184, 226, 152, 205, 245, 184, 199, 223, 87, 84, 27, 40, 156, 118, 140, 205, 59, 147, 149, 150, 126, 242, 243, 31, 95, 154, 235, 215, 76, 253, 222], [142, 222, 254, 223, 243, 246, 190, 246, 191, 207, 254, 183, 236, 255, 183, 229, 255, 127, 191, 199, 230, 223, 59, 255, 63, 127, 223, 131, 151, 117, 173, 255], [170, 184, 176, 152, 237, 245, 184, 207, 159, 95, 116, 27, 172, 156, 118, 175, 205, 27, 147, 145, 54, 111, 114, 241, 95, 89, 218, 235, 151, 76, 189, 222], [223, 122, 189, 254, 17

In [7]:
# convert google doc strings to numbers

data = all_faces_str[1:]
face_order = 0
for row in data:
    
    face_order += 1
    myTotal = np.array([])
    for element in row:
        myArray = np.array(eval(element))
        myTotal = np.vstack([myTotal,myArray]) if myTotal.size else myArray

        
    # convert data type
    myTotal_uint8 = np.uint8(myTotal) 
    
    # Add each image descriptor list from the database
    clusters = np.array([myTotal_uint8])
    bf.add(clusters)
    
    # Train: Does nothing for BruteForceMatcher though.
    bf.train()
    
    matches = bf.match(des1,myTotal_uint8)
    matches = sorted(matches, key = lambda x:x.distance)
    
    facest_face = 0
    numb_matches = (len(matches))
    if numb_matches > facest_face:
        facest_face = numb_matches
    
    face_found = False
    if facest_face > thres_hold:
        face_found = True
        found_face_order = face_order

    print(facest_face)
    print(face_found)

132
False
500
True


In [8]:
# Add face to database if not found, otherwise report the face

if face_found == True:
    print ("FACE FOUND! Face number:")
    print (found_face_order)
else:
    print ("Face NOT found. adding face to database")
    sheet.append_row('')
    col_in = 1
    row_in = sheet.row_count

    collumn_list = []
    des_cnt = 0
    des_per_cell = 100
    for collumn in des1:

        collumn_list.append(collumn.tolist())
        # Append des_per_cell # of collumns into a single collumn and upload
        if des_cnt >= des_per_cell -1:
            print ("cloud")
            sheet.update_cell(row_in, col_in, collumn_list)
            collumn_list = []
            col_in += 1
            des_cnt = 0
        else:
            des_cnt += 1

FACE FOUND! Face number:
2


# convert google doc strings to numbers
#all_faces_flt = list(map(float, all_faces))

print (type(all_faces_str))

#bf = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
data = all_faces_str[1:]
for row in data:
    myTotal = np.array([])
    #for element in data[0]:
    for element in row:
        myArray = np.array(eval(element))
        #print (myArray)
        myTotal = np.vstack([myTotal,myArray]) if myTotal.size else myArray

    print (myTotal)   

    print (des1.size)
    print (myTotal.size)
    print (des1)
    print (type(myTotal[0][0]))
    
    print ("des types")
    print (type(des2[0][0]))
    print (type(des1[0][0]))
        
    # convert data type
    myTotal_uint8 = np.uint8(myTotal) 
    print (type(myTotal_uint8[0][0]))
    
    
    #matches = bf.match(des1,myTotal)
    # create BFMatcher object
    #bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # Add each image descriptor list from the database
    clusters = np.array([myTotal_uint8])
    #clusters = np.array([des2])
    #clusters = np.array([myTotal])
    bf.add(clusters)
    
    # Train: Does nothing for BruteForceMatcher though.
    bf.train()
    
    print ("\nDES2")
    print (des2.size)
    print (des2)
    print ("\nReadDES2")
    print (myTotal_uint8.size)
    print (myTotal_uint8)
    print ("\nReadDES2 uncoverted")
    print (myTotal.size)
    print (myTotal)
    
    matches = bf.match(des1,myTotal_uint8)
    matches = sorted(matches, key = lambda x:x.distance)
    
    facest_face = 0
    numb_matches = (len(matches))
    if numb_matches > facest_face:
        facest_face = numb_matches


    face_found = False
    if facest_face > thres_hold:
        face_found = True

    #row_float = list(map(float,row))
    #all_faces_flt.append(row_float)

    facest_face
    print(facest_face)
    print(face_found)
    #print(all_faces_flt)
    
# convert data type
#myTotal_uint8 = np.arange(myTotal.size, dtype='uint8') 


#matches = bf.match(des1)

# Sort them in the order of their distance.
#matches = sorted(matches, key = lambda x:x.distance)



In [9]:
# Calculate the results
for tests in matches:
    numb_matches = (len(matches))
    if facest_face > numb_matches:
        facest_face = numb_matches


    if facest_face > thres_hold:
        face_found = True

#row_float = list(map(float,row))
#all_faces_flt.append(row_float)

facest_face
print(facest_face)
print(face_found)
#print(all_faces_flt)

500
True


# convert google doc strings to numbers
#all_faces_flt = list(map(float, all_faces))

print (type(all_faces_str))

data = all_faces_str[1:]

myTotal = np.array([])
for element in data[0]:
   myArray = np.array(eval(element))
   #print (myArray)
   myTotal = np.vstack([myTotal,myArray]) if myTotal.size else myArray

print (myTotal)   


#TODO remove after testing
all_faces_flt = []
face_found = False
facest_face = 0
for row in all_faces_str[1:]:
    
    print (type(row))
    print ("length of row")
    print (len(row))
    print (len(row[0]))
    print (len(row[0]+row[1]+row[2]+row[3]+row[4]))
    row_combined = row[0]+row[1]+row[2]+row[3]+row[4]
    
    print ("")
    print (len(row[0]))
    print (row[0])
    print ("")
    print (len(row[1]))
    print (row[1])
    print ("")
    print (len(row[2]))
    print (row[2])
    print ("")
    print (len(row[3]))
    print (row[3])
    print ("")
    print (len(row[4]))
    print (row[4])
    
    print ("")
    print (row_combined)
    
    
 
    #print (row)
    
    row_list = []
    for i in range (0, len(row)-1):
        row_list+= row[i]
    print ("")
    print (row_list)
    print(len(row_list))
    
    
    
    #np_row = np.array(row)
    
    
    
    #print(np_row.size)
    #print(np_row)
    #row_reconstructed = np_row.flatten()
    #row_reconstructed = np.hstack(np_row)
    #print (row_reconstructed.size)
    #print (row_reconstructed)
    
    #print (np_row[0])
    #print (np_row.shape)
    
    #row_reconstructed = np_row[0]
    #brute force it
    #row_reconstructed = np.concatenate(np_row[0],np_row[1],np_row[2],np_row[3],np_row[4])
    
    
    #for collumn in np_row[:1]:
    #    np_collumn = np.array(collumn)
    #    print(np_collumn)
    ##    row_reconstructed = np.concatenate((row_reconstructed,np_collumn))  
    
    #print (row_reconstructed.shape)
    #print (row_reconstructed.size)
    #print (row_reconstructed)
    
    
    
    
    matches = bf.match(des1,row)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    numb_matches = (len(matches))
    if facest_face > numb_matches:
        facest_face = numb_matches
    
    
    if facest_face > thres_hold:
        face_found = True
    
    #row_float = list(map(float,row))
    #all_faces_flt.append(row_float)
    
facest_face
print(facest_face)
print(face_found)
#print(all_faces_flt)

In [10]:
# Final result from the face detection characterizaion list is
# all_faces_flt
# Use where needed

In [11]:
###########################################
# Write to the Google doc
###########################################

In [12]:
# dumby data converted to text. Set "face_data" to the row you wish to append
face_data = [4.5, 6, 8, 2.133, 35623, 8]

In [13]:
# convert the float array to a string array before adding to the google doc
row = list(map(str, face_data))
print(row) 

['4.5', '6', '8', '2.133', '35623', '8']


# insert a row
sheet.append_row(row)